In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

2024-08-09 04:42:59.029893: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
data = pd.read_csv('../data/combined_keypoints.csv')

print("First few rows of the dataset:")
print(data.head())

print(f"Dataset shape: {data.shape}")

label_encoder = LabelEncoder()
data['Class'] = label_encoder.fit_transform(data['Class'])
print("\nMapping of classes:")
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

X = data.drop(columns=['Class'])
y = data['Class']

num_classes = len(label_encoder.classes_)
y = to_categorical(y, num_classes=num_classes)

First few rows of the dataset:
        Class    Nose_x    Nose_y  Nose_conf  LeftEye_x  LeftEye_y  \
0  Warrior_II  0.275027  0.441496   0.838373   0.257330   0.454769   
1  Warrior_II  0.361404  0.488448   0.632348   0.354708   0.496576   
2  Warrior_II  0.226947  0.543324   0.787434   0.213398   0.533082   
3  Warrior_II  0.392478  0.374899   0.328839   0.381358   0.370187   
4  Warrior_II  0.223151  0.476531   0.887696   0.211527   0.493164   

   LeftEye_conf  RightEye_x  RightEye_y  RightEye_conf  ...  LeftKnee_conf  \
0      0.874609    0.258129    0.449623       0.896995  ...       0.941765   
1      0.715429    0.351345    0.491862       0.855811  ...       0.829439   
2      0.857410    0.214061    0.525327       0.864843  ...       0.923154   
3      0.546548    0.383478    0.369107       0.577419  ...       0.324949   
4      0.910345    0.209859    0.482766       0.860895  ...       0.828863   

   RightKnee_x  RightKnee_y  RightKnee_conf  LeftAnkle_x  LeftAnkle_y  \
0     

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

print("\nShape of X_train after reshaping:", X_train.shape)
print("Shape of X_test after reshaping:", X_test.shape)


Shape of X_train after reshaping: (10981, 1, 51)
Shape of X_test after reshaping: (2746, 1, 51)


In [ ]:
def create_model():
    model = Sequential()

    model.add(Bidirectional(LSTM(units=512, return_sequences=True, kernel_regularizer=l2(0.01)), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(units=256, return_sequences=True, kernel_regularizer=l2(0.01))))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(units=128, kernel_regularizer=l2(0.01))))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(units=256, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(units=128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Dense(units=64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.1))

    model.add(Dense(units=num_classes, activation='softmax'))

    optimizer = 'adam'
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-6, verbose=1)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1

for train_index, val_index in kf.split(X_train):
    print(f"\nTraining fold {fold_no}...")

    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    model = create_model()

    history = model.fit(
        X_train_fold, y_train_fold,
        epochs=150,
        batch_size=64,
        validation_data=(X_val_fold, y_val_fold),
        callbacks=[reduce_lr],
        verbose=1
    )

    fold_no += 1


Training fold 1...


2024-08-09 04:43:18.539359: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-08-09 04:43:19.604970: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-09 04:43:19.607873: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-09 04:43:19.6

Epoch 1/150


2024-08-09 04:43:22.888470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-09 04:43:22.892993: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-09 04:43:22.896767: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

138/138 [==============================] - ETA: 0s - loss: 20.1383 - accuracy: 0.1916

...

Epoch 1/150


2024-08-09 08:09:14.258704: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-09 08:09:14.263155: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-09 08:09:14.266847: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

138/138 [==============================] - ETA: 0s - loss: 20.0392 - accuracy: 0.1923

2024-08-09 08:09:50.644948: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-09 08:09:50.649002: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-09 08:09:50.653024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

138/138 [==============================] - 41s 158ms/step - loss: 20.0392 - accuracy: 0.1923 - val_loss: 12.0987 - val_accuracy: 0.0132 - lr: 0.0010
Epoch 2/150
138/138 [==============================] - 20s 141ms/step - loss: 7.2045 - accuracy: 0.4212 - val_loss: 8.0372 - val_accuracy: 0.0132 - lr: 0.0010
Epoch 3/150
138/138 [==============================] - 19s 140ms/step - loss: 4.2243 - accuracy: 0.4963 - val_loss: 6.1090 - val_accuracy: 0.0587 - lr: 0.0010
Epoch 4/150
138/138 [==============================] - 19s 138ms/step - loss: 3.2335 - accuracy: 0.5274 - val_loss: 4.3891 - val_accuracy: 0.2077 - lr: 0.0010
Epoch 5/150
138/138 [==============================] - 20s 142ms/step - loss: 2.8474 - accuracy: 0.5341 - val_loss: 3.0751 - val_accuracy: 0.4426 - lr: 0.0010
Epoch 6/150
138/138 [==============================] - 20s 144ms/step - loss: 2.5764 - accuracy: 0.5629 - val_loss: 3.2187 - val_accuracy: 0.3534 - lr: 0.0010
Epoch 7/150
138/138 [==============================] - 1

In [ ]:
val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
print(f"Validation accuracy for fold {fold_no - 1}: {val_accuracy * 100:.2f}%")

Validation accuracy for fold 5: 81.38%


In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
test_accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f'\nFinal test accuracy: {test_accuracy * 100:.2f}%')

2024-08-09 14:48:37.400310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-09 14:48:37.404472: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-09 14:48:37.407859: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

86/86 [==============================] - 6s 13ms/step

Final test accuracy: 81.39%


In [ ]:
final_loss, final_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'\nFinal test accuracy: {final_accuracy * 100:.2f}%')


Final test accuracy: 81.39%


In [ ]:
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Classification Report:


NameError: name 'y_test' is not defined